In [ ]:
import sys
sys.path.append('../../common')
import utils
import importlib
importlib.reload(utils)

from utils import IOHandler as IO

import time

import numpy as np
import pynput.keyboard as kb

from enum import Enum

io = utils.IOHandler(offset=(880, 558), game_dims=(1600,1200), verbose=True)

In [ ]:
class Screen(Enum):
    GAME_OVER = 'game_over'
    HOME = 'home'
    MIDGAME = 'midgame'
    GREAT_PLAY = "great_play"

SCREEN_TYPES = {
    Screen.GAME_OVER: [790, 1140, [125, 251, 76]], # bright green border
    Screen.HOME: [70, 965, [194, 41, 125]], # pink mushroom
    Screen.MIDGAME: [1330, 60, [179, 105, 247]], # purple music note
    Screen.GREAT_PLAY: [700, 1165, [239, 140, 52]],
}

class Tile(Enum):
    UNKOWN = 'unkown'
    ANT_UP = 'ant_up'
    ANT_DOWN = 'ant_down'
    ANT_RIGHT = 'ant_right'
    ANT_LEFT = 'ant_left'
    ANT_TR = 'ant_tr'
    ANT_TL = 'ant_tl'
    ANT_BR = 'ant_br'
    ANT_BL = 'ant_bl'
    BLANK_LBLUE = 'blank_lblue'
    BLANK_WHITE = 'blank_white'
    BLANK_DBLUE = 'blank_dblue'
    FOOD_TOMATO = 'food_tomato'
    FOOD_CHICKEN = 'food_chicken'
    FOOD_BANANA = 'food_banana'
    FOOD_HOTDOG = 'food_hotdog'
    FOOD_PEAR = 'food_pear'
    FOOD_SANDWICH = 'food_sandwich'
    FOOD_SODA = 'food_soda'
    FOOD_YOGURT = 'food_yogurt'
    FOOD_STRAWBERRY = 'food_strawberry'
    FOOD_LOLLIPOP = 'food_lollipop'
    FOOD_PIZZA = 'food_pizza'
    FOOD_CAKE = 'food_cake'

tile_ids = {
    Tile.ANT_UP: ["ant_up.png"],
    Tile.ANT_DOWN: ["ant_down.png"],
    Tile.ANT_RIGHT: ["ant_right.png"],
    Tile.ANT_LEFT: ["ant_left.png"],
    Tile.ANT_TR: ["ant_top_right.png"],
    Tile.ANT_TL: ["ant_top_left.png"],
    Tile.ANT_BR: ["ant_bottom_right.png"],
    Tile.ANT_BL: ["ant_bottom_left.png"],
    Tile.BLANK_LBLUE: ["blank_lblue.png"],
    Tile.BLANK_DBLUE: ["blank_dblue.png"],
    Tile.BLANK_WHITE: ["blank_white.png"],
    Tile.FOOD_TOMATO: ["tomato.png"],
    Tile.FOOD_CHICKEN: ["chicken.png"],
    Tile.FOOD_BANANA: ["banana_white.png", "banana_lblue.png", "banana_dblue.png"],
    Tile.FOOD_HOTDOG: ["hotdog.png"],
    Tile.FOOD_PEAR: ["pear.png"],
    Tile.FOOD_SANDWICH: ["sandwich.png"],
    Tile.FOOD_SODA: ["soda.png"],
    Tile.FOOD_YOGURT: ["yogurt.png"],
    Tile.FOOD_STRAWBERRY: ["strawberry.png"],
    Tile.FOOD_LOLLIPOP: ["lollipop_lblue_right.png", "lollipop_lblue_left.png","lollipop_dblue_right.png", "lollipop_dblue_left.png","lollipop_white_right.png", "lollipop_white_left.png"],
    Tile.FOOD_PIZZA: ["pizza.png"],
    Tile.FOOD_CAKE: ["cake.png"]
}

def get_best_tile_info(tile_ids, img):
    best_dist = -1
    best_type = Tile.UNKOWN
    best_file = None
    best_mask = None
    for tile_type, masks in tile_ids.items():
        for filename, mask in masks.items():
            dist = np.mean(np.sqrt(np.sum(np.square(img - mask), axis = 2)))
            if best_dist == -1 or dist < best_dist:
                best_dist = dist
                best_type = tile_type
                best_file = filename
                best_mask = mask
    return best_type, best_file, best_dist, best_mask

def get_tile_type(tile_center):
    best_type, best_file, best_dist, best_mask = get_best_tile_info(tile_ids, tile_center)
    if best_dist > MAX_DIST:
        best_type = Tile.UNKOWN
    return best_type

tile_char = {
    Tile.ANT_UP: "↑",
    Tile.ANT_DOWN: "↓",
    Tile.ANT_RIGHT: "→",
    Tile.ANT_LEFT: "←",
    Tile.ANT_TR: "↗",
    Tile.ANT_TL: "↖",
    Tile.ANT_BR: "↘",
    Tile.ANT_BL: "↙",
    Tile.FOOD_TOMATO: "🍅",
    Tile.FOOD_CHICKEN: "🍗",
    Tile.FOOD_BANANA: "🍌",
    Tile.FOOD_HOTDOG: "🌭",
    Tile.FOOD_PEAR: "🍐",
    Tile.FOOD_SANDWICH: "🥪",
    Tile.FOOD_SODA: "🥤",
    Tile.FOOD_YOGURT: "🥣",
    Tile.FOOD_STRAWBERRY: "🍓",
    Tile.FOOD_LOLLIPOP: "🍭",
    Tile.FOOD_PIZZA: "🍕",
    Tile.FOOD_CAKE: "🍰",
    Tile.BLANK_LBLUE: "▒",
    Tile.BLANK_WHITE: "░",
    Tile.BLANK_DBLUE: "▓",
    Tile.UNKOWN: "❓"
}

def remove_background(bg, img):
    img.copy()
    img[(img == bg).all(axis=2)] *= 0
    return img

def is_food(tile_type):
    return tile_type.name[:4] == "FOOD"

def is_ant(tile_type):
    return tile_type.name[:3] == "ANT"

def is_blank(tile_type):
    return tile_type.name[:5] == "BLANK"

def is_unknown(tile_type):
    return tile_type == Tile.UNKNOWN

def capture_tile(i, j):
    x = i*CELL_DIMS[0]
    y = j*CELL_DIMS[1]
    w = CELL_DIMS[0]
    h = CELL_DIMS[1]
    return io.capture_portion(x, y, w, h)

def capture_tile_center(i, j, size = 8):
    x = BLANKET_OFFSET[0] + i*CELL_DIMS[0] + (52 - size) // 2
    y = BLANKET_OFFSET[1] + j*CELL_DIMS[1] + (52 - size) // 2
    return io.capture_portion(x, y, size, size)

hamiltonian_path = {
    'zig-zag': [
        (np.array([13, 17]), [kb.Key.left, kb.Key.up]),
        (np.array([12, 1]), [kb.Key.left, kb.Key.down]),
        (np.array([11, 17]), [kb.Key.left, kb.Key.up]),
        (np.array([10, 1]), [kb.Key.left, kb.Key.down]),
        (np.array([9, 17]), [kb.Key.left, kb.Key.up]),
        (np.array([8, 1]), [kb.Key.left, kb.Key.down]),
        (np.array([7, 17]), [kb.Key.left, kb.Key.up]),
        (np.array([6, 1]), [kb.Key.left, kb.Key.down]),
        (np.array([5, 17]), [kb.Key.left, kb.Key.up]),
        (np.array([4, 1]), [kb.Key.left, kb.Key.down]),
        (np.array([3, 17]), [kb.Key.left, kb.Key.up]),
        (np.array([2, 1]), [kb.Key.left, kb.Key.down]),
        (np.array([1, 17]), [kb.Key.left, kb.Key.up]),
        (np.array([0, 0]), [kb.Key.right, kb.Key.right]),
        (np.array([25, 0]), [kb.Key.down, kb.Key.down]),
        (np.array([25, 17]), [kb.Key.left, kb.Key.up]),
        (np.array([24, 1]), [kb.Key.left, kb.Key.down]),
        (np.array([23, 17]), [kb.Key.left, kb.Key.up]),
        (np.array([22, 1]), [kb.Key.left, kb.Key.down]),
        (np.array([21, 17]), [kb.Key.left, kb.Key.up]),
        (np.array([20, 1]), [kb.Key.left, kb.Key.down]),
        (np.array([19, 17]), [kb.Key.left, kb.Key.up]),
        (np.array([18, 1]), [kb.Key.left, kb.Key.down]),
        (np.array([17, 17]), [kb.Key.left, kb.Key.up]),
        (np.array([16, 1]), [kb.Key.left, kb.Key.down]),
        (np.array([15, 17]), [kb.Key.left, kb.Key.up]),
        (np.array([14, 1]), [kb.Key.left, kb.Key.down])
    ],
    'spacious': [
        (np.array([13, 16]), [kb.Key.left, kb.Key.up]),
        (np.array([12, 2]), [kb.Key.left, kb.Key.down]),
        (np.array([11, 16]), [kb.Key.left, kb.Key.up]),
        (np.array([10, 2]), [kb.Key.left, kb.Key.down]),
        (np.array([9, 16]), [kb.Key.left, kb.Key.up]),
        (np.array([8, 2]), [kb.Key.left, kb.Key.down]),
        (np.array([7, 16]), [kb.Key.left, kb.Key.up]),
        (np.array([6, 2]), [kb.Key.left, kb.Key.down]),
        (np.array([5, 16]), [kb.Key.left, kb.Key.up]),
        (np.array([4, 2]), [kb.Key.left, kb.Key.down]),
        (np.array([3, 16]), [kb.Key.left, kb.Key.up]),
        (np.array([2, 2]), [kb.Key.left, kb.Key.down]),
        (np.array([1, 16]), [kb.Key.left, kb.Key.up]),
        (np.array([0, 0]), [kb.Key.right, kb.Key.right]),
        (np.array([25, 0]), [kb.Key.down, kb.Key.down]),
        (np.array([25, 16]), [kb.Key.left, kb.Key.up]),
        (np.array([24, 2]), [kb.Key.left, kb.Key.down]),
        (np.array([23, 16]), [kb.Key.left, kb.Key.up]),
        (np.array([22, 2]), [kb.Key.left, kb.Key.down]),
        (np.array([21, 16]), [kb.Key.left, kb.Key.up]),
        (np.array([20, 2]), [kb.Key.left, kb.Key.down]),
        (np.array([19, 16]), [kb.Key.left, kb.Key.up]),
        (np.array([18, 2]), [kb.Key.left, kb.Key.down]),
        (np.array([17, 16]), [kb.Key.left, kb.Key.up]),
        (np.array([16, 2]), [kb.Key.left, kb.Key.down]),
        (np.array([15, 16]), [kb.Key.left, kb.Key.up]),
        (np.array([14, 2]), [kb.Key.left, kb.Key.down])
    ],
    'spiral': [
        (np.array([13, 17]), [kb.Key.right]),
        (np.array([25, 17]), [kb.Key.up]),
        (np.array([25, 0]), [kb.Key.left]),
        (np.array([0, 0]), [kb.Key.down, kb.Key.right]),
        # (np.array([0, 0]), [kb.Key.down]),
        # (np.array([0, 1]), [kb.Key.right]),
        (np.array([24, 1]), [kb.Key.down]),

        (np.array([24, 16]), [kb.Key.left]),
        (np.array([1, 16]), [kb.Key.up]),
        (np.array([1, 3]), [kb.Key.right]),
        (np.array([22, 3]), [kb.Key.down]),

        (np.array([22, 14]), [kb.Key.left]),
        (np.array([3, 14]), [kb.Key.up]),
        (np.array([3, 5]), [kb.Key.right]),
        (np.array([20, 5]), [kb.Key.down]),

        (np.array([20, 12]), [kb.Key.left]),
        (np.array([5, 12]), [kb.Key.up]),
        (np.array([5, 7]), [kb.Key.right]),
        (np.array([18, 7]), [kb.Key.down]),

        (np.array([18, 10]), [kb.Key.left]),
        (np.array([7, 10]), [kb.Key.up, kb.Key.right]),
        # (np.array([7, 10]), [kb.Key.up]),
        # (np.array([7, 9]), [kb.Key.right]),
        (np.array([17, 9]), [kb.Key.up, kb.Key.left]),
        # (np.array([17, 9]), [kb.Key.up]),
        # (np.array([17, 8]), [kb.Key.left]),

        (np.array([6, 8]), [kb.Key.down]),
        (np.array([6, 11]), [kb.Key.right]),
        (np.array([19, 11]), [kb.Key.up]),

        (np.array([19, 6]), [kb.Key.left]),
        (np.array([4, 6]), [kb.Key.down]),
        (np.array([4, 13]), [kb.Key.right]),
        (np.array([21, 13]), [kb.Key.up]),

        (np.array([21, 4]), [kb.Key.left]),
        (np.array([2, 4]), [kb.Key.down]),
        (np.array([2, 15]), [kb.Key.right]),
        (np.array([23, 15]), [kb.Key.up]),

        (np.array([23, 2]), [kb.Key.left]),
        (np.array([0, 2]), [kb.Key.down]),
        (np.array([0, 17]), [kb.Key.right]),
    ]
}

dir_offset = {
    kb.Key.right: np.array([1, 0]),
    kb.Key.left: np.array([-1, 0]),
    kb.Key.up: np.array([0, -1]),
    kb.Key.down: np.array([0, 1])
}

# Excellent for medium difficulty
def quick_u_turn(keys, difficulty):
    if difficulty == 'easy':
        delays = [0.1, 0.05, 0.05, 0]
    elif difficulty == 'medium':
        delays = [0.05, 0.05, 0.05, 0]
    elif difficulty == 'hard':
        # delays = [0.05, 0.05, 0.05, 0]
        delays = [0.03, 0.05, 0.05, 0]
    if len(keys) == 1:
        keys = keys*2
    io.press_key(keys[0], delays[0])
    io.press_key(keys[1], delays[1])
    io.release_key(keys[0], delays[2])
    io.release_key(keys[1], delays[3])

def quick_is_ant(i, j):
    tile_center = capture_tile_center(i, j, size = MASK_SIZE)
    dist = IO.calc_image_dist(tile_center, avg_ant_mask, difficulty)
    return dist < 70

def get_cell_in_direction(pos, direction):
    return pos + dir_offset[direction]

# Get into the main game
# TODO: move to utils?
def get_into_game(difficulty='easy'):
  global break_program
  new_game = False
  while True:
    if break_program:
      break
    screen_types = io.calc_screen_types()
    print(screen_types)

    if Screen.GREAT_PLAY in screen_types:
        io.click_mouse(800, 1100) # "Collect" button
        new_game = True
    elif Screen.GAME_OVER in screen_types:
        io.click_mouse(595, 895) # "Replay" button
        new_game = True
    elif Screen.HOME in screen_types:
        # Start game with given difficulty
        if difficulty == 'easy':
            io.click_mouse(400, 990)
        elif difficulty == 'medium':
            io.click_mouse(800, 990)
        elif difficulty == 'hard':
            io.click_mouse(1200, 990)
        io.click_mouse(0, 0)
        time.sleep(1.5)
        new_game = True
    elif Screen.MIDGAME in screen_types:
        return screen_types, new_game
    time.sleep(0.2)

# TODO: move break/kill program to utils and make names/keys consistent in bots
def on_press(key):
    global break_program, listener
    if key == kb.Key.esc:
        print("Escape pressed to quit")
        listener.stop()
        break_program = True

def execute_turns_after_trigger(trigger_cell, directions, avg_ant_mask, difficulty):
    if len(directions) == 2:
        return _execute_u_turn_after_trigger(trigger_cell, directions, avg_ant_mask, difficulty)
    if len(directions) == 1:
        return _execute_turn_after_trigger(trigger_cell, directions[0], avg_ant_mask, difficulty)
    raise Exception('Can only execute 1-2 turns!')

def _execute_u_turn_after_trigger(trigger_cell, directions, avg_ant_mask, difficulty):
    trigger_pos = BLANKET_OFFSET + 25 + trigger_cell*CELL_DIMS 
    next_cell = get_cell_in_direction(trigger_cell, directions[0])
    next_pos = BLANKET_OFFSET + 25 + next_cell*CELL_DIMS 
    final_cell = get_cell_in_direction(next_cell, directions[1])
    final_pos = BLANKET_OFFSET + 25 + final_cell*CELL_DIMS 
    print(f'U-TURN: heading for trigger {trigger_cell}, where we\'ll press {directions} to reach {next_cell} then {final_cell}')

    hit_trigger_cell = __wait_to_hit_trigger_pos(trigger_pos, avg_ant_mask)
    if not hit_trigger_cell:
        return False
    print(f'Hit trigger cell: {trigger_cell}, do a quick u turn through {next_cell}')
    hit_next_cell = __execute_turns_now(next_pos, directions, avg_ant_mask, difficulty)
    if not hit_next_cell:
        return False
    # TODO: can maybe remove, this was to check if we hit the final pos but whatever
    print(f'Tried to u-turn through {next_cell}, check if hit final_cell {final_cell}')
    hit_final_cell = __wait_to_hit_trigger_pos(final_pos, avg_ant_mask)
    if not hit_final_cell:
        return False
    print(f'SUCCESS! Hit final cell {final_cell} of u-turn')
    return True

def _release_directions(directions):
    print(f'Release {directions}') 
    [io.release_key(direction) for direction in directions]

# Start repeatedly hammering direction, until we hit the next_pos, return whether successful
# If a u-turn, quick_u_turn and hope that it works because there is zero room for error
def __execute_turns_now(next_pos, directions, avg_ant_mask, difficulty):
    global break_program
    presses = 0
    while not break_program:
        if pixel_is_ant(next_pos, avg_ant_mask):
            if presses > 0:
                _release_directions(directions)
            return True
        if len(directions) == 2:
            print(f'Click quick u-turn and hope: {directions}')
            quick_u_turn(directions, difficulty)
            return True
        else:
            print(f'Press {directions[0]}')
            io.press_key(directions[0])
        presses += 1
        if presses > 5:
            print(f'Hit the trigger, but turn not successful after {presses} attempts, give up')
            _release_directions(directions)
            return False 
    _release_directions(directions)
    return False

def pixel_is_ant(pos, avg_ant_mask):
    pixel = io.capture_pixel(pos[0], pos[1])
    dist = IO.calc_color_dist(pixel, avg_ant_mask)
    return dist < 64

def __wait_to_hit_trigger_pos(trigger_pos, avg_ant_mask, max_wait=7):
    last_update = time.time()
    while not break_program:
        if pixel_is_ant(trigger_pos, avg_ant_mask):
            return True
        if time.time() - last_update > max_wait:
            print(f"Haven't hit the trigger after {max_wait} seconds, give up")
            return False
    return False

def _execute_turn_after_trigger(trigger_cell, direction, avg_ant_mask, difficulty):
    trigger_pos = BLANKET_OFFSET + 25 + trigger_cell*CELL_DIMS 
    next_cell = get_cell_in_direction(trigger_cell, direction)
    next_pos = BLANKET_OFFSET + 25 + next_cell*CELL_DIMS 
    print(f'Heading for trigger {trigger_cell}, where we\'ll turn {direction} to reach {next_cell}')
    hit_trigger_cell = __wait_to_hit_trigger_pos(trigger_pos, avg_ant_mask)
    if not hit_trigger_cell:
        return False
    turn_success = __execute_turns_now(next_pos, [direction], avg_ant_mask, difficulty)
    return turn_success

In [ ]:
# TODO: add infinite game play
io.set_screen_types(SCREEN_TYPES)

break_program = False
listener = kb.Listener(on_press=on_press)
listener.start()

io.click_mouse(0, 0)

GRID_DIMS = np.array([26, 18])
CELL_DIMS = np.array([52, 52])
BLANKET_OFFSET = np.array([125, 136])
BLANKET_DIMS = np.array([1352, 936])
MAX_DIST = 100

difficulty = 'medium'
# path_shape = 'zig-zag'
path_shape = 'spiral'
# path_shape = 'spacious'

avg_ant_mask = io.load_image("no-commit/unique_tiles/best_choices/ant.png")[3, 3, :]

step = 0
# num_caps = 0
# start = time.time()
print(f'avg_ant_mask: {avg_ant_mask}')
while not break_program:
    _, new_game = get_into_game(difficulty)
    if new_game:
        print("Started a new game")
        step = 0
    trigger_cell, directions = hamiltonian_path[path_shape][step % len(hamiltonian_path[path_shape])]
    turns_success = execute_turns_after_trigger(trigger_cell, directions, avg_ant_mask, difficulty)
    if turns_success:
        step += 1
    else:
        step = 0
    
# duration = time.time() - start
# print(f'{num_caps} screenshots in {duration:.2f} seconds = {num_caps/duration:.2f} screenshots per second')
listener.stop()

In [ ]:
# # Load our best masks for each tile type
# for tile_type, files in tile_ids.items():
#     imgs = {}
#     for file in files:
#         imgs[file] = load_image(f"unique_tiles/best_choices/{file}")
#     tile_ids[tile_type] = imgs

# # Try subtracting some the blanket background from an image
# background = load_image("level_bg.png")
# significant = load_image("saved/2021-07-11 14.57.11.png")
# show_image(remove_background(background, significant))

# # Parse the 'tile' types from a game image
# blanket = capture_portion(BLANKET_OFFSET[0], BLANKET_OFFSET[1],BLANKET_DIMS[0],BLANKET_DIMS[1])
# show_image(blanket)
# grid_tiles = [[0 for j in range(GRID_DIMS[1])] for i in range(GRID_DIMS[0])]
# for i in range(GRID_DIMS[0]):
#     for j in range(GRID_DIMS[1]):
#         x = i*CELL_DIMS[0]
#         y = j*CELL_DIMS[1]
#         w = CELL_DIMS[0]
#         h = CELL_DIMS[1]
#         tile_img = blanket[x:x+w, y:y+h, :]
#         grid_tiles[i][j] = get_tile_type(tile_img)
# for row in np.array(grid_tiles).T.tolist():
#     print(' '.join([tile_char[x] for x in row]))

# # Get the average image for each tile type
# distances = {}
# for folder in os.walk("unique_tiles"):
#     dir_path, _, filenames = folder
#     if dir_path == "unique_tiles":
#         continue
#     saved = set()
#     tile_images = {}
#     tiles = []
#     for filename in filenames:
#         if filename[-4:] != ".png":
#             continue
#         tile_mid = load_image(f"{dir_path}/{filename}")
#         tile_images[filename] = tile_mid
#         tiles.append(tile_mid)
#
#     distances[f"{dir_path}"] = {}
#
#     for i, tile_1 in enumerate(list(tile_images.keys())):
#         tile_img_1 = tile_images[tile_1]
#         max_dist = 0
#         for j, tile_2 in enumerate(list(tile_images.keys())):
#             if i == j:
#                 continue
#             tile_img_2 = tile_images[tile_2]
#             distance = image_dist(tile_img_1, tile_img_2)
#             if distance > max_dist:
#                 max_dist = distance
#         distances[f"{dir_path}"][tile_1] = max_dist
#
#     avg_tile = np.mean(np.array(tiles), axis=0)
#     save_image(avg_tile, f"{dir_path}/average.png")

# # Get the image in the folder that best matches the average?
# best_files = {}
# for folder in distances:
#     min_dist = 1000000000000
#     best_file = ""
#     for file in distances[folder]:
#         dist = distances[folder][file]
#         if dist < min_dist:
#             min_dist = dist
#             best_file = file
#     best_files[folder] = {best_file: min_dist}

# # Save the best choices
# output = []
# for folder, item in best_files.items():
#     if folder == "unique_tiles/best_choices":
#         continue
#     filename, dist = list(item.items())[0]
#     save_image(load_image(f"{folder}/{filename}"), f"unique_tiles/best_choices/{folder.split('/')[1]}.png")
#     output.append([folder, filename, dist])
#     print(filename)
#
# output = sorted(output, key = lambda l:l[2], reverse=True)
# print('\n'.join(["\t".join([str(r) for r in row]) for row in output]))

# # Load the best choices
# tile_ids = {}
# for file in os.listdir("unique_tiles/best_choices"):
#     if file[-4:] != ".png":
#         continue
#     tile_ids[file] = load_image(f"unique_tiles/best_choices/{file}")

# # Attempts to classify tile images using best mask
# max_dist = 0
# for folder in os.walk("unique_tiles"):
#     dir_path, _, filenames = folder
#     if dir_path == "unique_tiles" or dir_path == "unique_tiles/best_choices":
#         continue
#     for filename in filenames:
#         if filename[-4:] != ".png":
#             continue
#         tile = load_image(f"{dir_path}/{filename}")
#         best, dist = get_best_mask_dict(tile, tile_ids)
#         if (folder[0].split('/')[1] != best[:-4]):
#             print(f"{dir_path}/{filename}", best, dist, sep = '\t')
#         if dist > max_dist:
#             max_dist = dist
# print(max_dist)
# save_image(avg_tile, f"{dir_path}/average.png")

# # Saves a bunch of tile images, trying to dedupe with a hash
# k = 0
# while True:
#     blanket = capture_portion(BLANKET_OFFSET[0], BLANKET_OFFSET[1],BLANKET_DIMS[0],BLANKET_DIMS[1])
#     k += 1
#     grid_tiles = [[0 for j in range(GRID_DIMS[1])] for i in range(GRID_DIMS[0])]
#     for i in range(GRID_DIMS[0]):
#         for j in range(GRID_DIMS[1]):
#             x = i*CELL_DIMS[0]
#             y = j*CELL_DIMS[1]
#             w = CELL_DIMS[0]
#             h = CELL_DIMS[1]
#             tile_img = blanket[x:x+w, y:y+h, :]
#             part_to_hash = tile_img[22:30, 22:30, :]
#             # name = imagehash.average_hash(Image.fromarray(part_to_hash.swapaxes(0, 1).astype(np.uint8), "RGB"))
#             # name = imagehash.colorhash(Image.fromarray(part_to_hash.swapaxes(0, 1).astype(np.uint8), "RGB"))
#             name = hash(str(part_to_hash))
#             if name not in saved:
#                 saved.add(name)
#                 # name = f"unique_tiles/{name}.png"
#                 name = f"numpy_tiles/{name}.png"
#                 # name = f"color_hash/{name}.png"
#                 save_image(part_to_hash, name)
#                 print(f"saved {name}")
#     time.sleep(3)
#     for i in range(10):
#         print(f"{i}{'-'*i}")
#         time.sleep(0.01)
#     # grid_tiles[i][j] = get_tile_type(tile_img)

# # See how close lollipop and average ant are
# test = load_image("unique_tiles/best_choices/lollipop_dblue_right.png")[3, 3, :]
# print(color_dist(avg_ant_mask, test))

# # Tries to parse the grid
# grid_tiles = [[0 for j in range(GRID_DIMS[1])] for i in range(GRID_DIMS[0])]
# for i in range(GRID_DIMS[0]):
#     for j in range(GRID_DIMS[1]):
#         tile_center = capture_tile_center(i, j)
#         grid_tiles[i][j] = get_tile_type(tile_center)
# 
# for row in np.array(grid_tiles).T.tolist():
#     print(' '.join([tile_char[x] for x in row]))